In [ ]:
#pip install pydub

In [ ]:
from pydub import AudioSegment
import wave
import struct
from scipy import fromstring,int16
import numpy as np
import os
import math


class Prepare:
  def __init__(self,data_path,test_path,cut_time):
    self.data_path=data_path
    self.test_path=test_path
    self.cut_time=cut_time

    self.st_to_mn(data_path,test_path)

    
    # 一応既に同じ名前のディレクトリがないか確認。
    #音源分割用のフォルダ
    self.file = os.path.exists("/content/drive/MyDrive/data_output/")
    if self.file == False:
      #保存先のディレクトリの作成
      os.mkdir("/content/drive/MyDrive/data_output/")

    #鼻歌分割用のフォルダ
    self.file = os.path.exists("/content/drive/MyDrive/test_output/")
    if self.file == False:
      os.mkdir("/content/drive/MyDrive/test_output/")
    
    data_min='/content/drive/MyDrive/data_output/'
    test_min='/content/drive/MyDrive/test_output/'

    #self.data_min = data_min
    #self.test_min = test_min

    self.mono_data_path=self.data_path[:-4]+'_mono.wav'
    self.mono_test_path=self.test_path[:-4]+'_mono.wav'
    
    self.cut_wav(self.mono_data_path,float(self.cut_time),data_min)
    self.cut_wav(self.mono_test_path,float(self.cut_time),test_min)
    

  def st_to_mn(self,data_path,test_path):
    #音源をモノラルに変換
    sound = AudioSegment.from_wav(data_path)
    sound = sound.set_channels(1)
    sound.export(data_path[:-4]+'_mono.wav', format="wav")#音源のモノラルのパスを指定

    #鼻歌をモノラルに変換
    sound = AudioSegment.from_wav(test_path)
    sound = sound.set_channels(1)
    sound.export(test_path[:-4]+'_mono.wav', format="wav")#鼻歌のモノラルのパスを指定
  
  #ファイルを分割する
  #filenameに読み込むファイル、timeにカットする間隔
  def cut_wav(self,filename,time,output_folder):  
      # timeの単位は[sec]

      # ファイルを読み出し
      wavf = filename 
      wr = wave.open(wavf, 'r')

      # waveファイルが持つ性質を取得
      ch = wr.getnchannels()
      width = wr.getsampwidth()
      fr = wr.getframerate()
      fn = wr.getnframes()
      total_time = 1.0 * fn / fr 
      integer = math.floor(total_time*100) # 小数点以下切り捨て
      t = int(time*100)  # 秒数[sec]
      frames = int(ch * fr * t /100)
      num_cut = int(integer//t)
      # waveの実データを取得し、数値化
      data = wr.readframes(wr.getnframes())
      wr.close()
      X = np.frombuffer(data, dtype=int16)

      for i in range(num_cut):
        #print(i)
        # 出力データを生成
        outf = output_folder + str(10000+i) + '.wav' 
        start_cut = int(i*frames)
        end_cut = int(i*frames + frames)
        #print(start_cut)
        #print(end_cut)
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, 'w')
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()
